In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard library imports.
import datetime
import itertools
import json

# Related third party imports.
import numpy as np
import pandas as pd
import tqdm

# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation
from src.data.json_encoder import NumpyEncoder

In [11]:
network_name = '25_italy'
max_vSDN_size = 6
hp_type = 'ilp'
hp_objective = 'acceptance ratio'

possible_settings = {
    'network_name': [network_name],
    'latency_factor': np.arange(0.4, 0.7, 0.1),
    'shortest_k': [16],
    'hp_type': [hp_type],
    'hp_objective': [hp_objective],
    'hp_repeat':[4],
    'repeat': [100],
    'max_request_size': [int(max_vSDN_size/2)],
    'vSDN_count_ilp' : [10],
}
param_names_1 = list(possible_settings.keys())
setting_generator = [
    dict(zip(param_names_1, x))
    for x in itertools.product(*possible_settings.values())
]

possible_request_settings = {
    'request_size': range(2, max_vSDN_size),
    'count': [100]
}
# param_names_2 = list(possible_request_settings.keys())
# request_setting_generator = [
#     dict(zip(param_names_2, x))
#     for x in itertools.product(*possible_request_settings.values())
# ]

In [17]:
# Run
setting = setting_generator[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.run_static_simulation(possible_request_settings=possible_request_settings, **setting)
run_log = ns.get_logs()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 75 rows, 8775 columns and 1358 nonzeros
Model fingerprint: 0x7cef9075
Model has 7550 general constraints
Variable types: 0 continuous, 8775 integer (8775 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve added 1353 rows and 0 columns
Presolve removed 0 rows and 7566 columns
Presolve time: 0.06s
Presolved: 1428 rows, 1209 columns, 5736 nonzeros
Variable types: 0 continuous, 1209 integer (1209 binary)
Found heuristic solution: objective 22.0000000

Root relaxation: objective 7.194444e+00, 861 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.19444    0  226   22.00000 

In [18]:
# run_log = ns.log_simulation()
pd.DataFrame(run_log)[['vSDN_size', 'acceptance_ratio']].groupby(by='vSDN_size').mean()

,acceptance_ratio
vSDN_size,
2,0.919118
3,0.709677
4,0.582500
5,0.392500


In [ ]:
# Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.run_simulation(possible_request_settings=possible_request_settings, **setting)
    simulation_logs.extend(ns.get_logs())

In [ ]:
# Baseline Run
setting = setting_generator[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.run_optimal_static_simulation(possible_request_settings=possible_request_settings, **setting)
run_log = ns.get_logs()

In [16]:
pd.DataFrame(run_log)[['vSDN_size', 'acceptance_ratio']].groupby(by='vSDN_size').mean()

,acceptance_ratio
vSDN_size,
2,0.941176
3,0.822581
4,0.697500
5,0.510000


In [6]:
# Baseline Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.baseline_simulation(possible_request_settings=possible_request_settings, **setting)
    simulation_logs.extend(ns.get_logs())

  0%|          | 0/3 [00:00<?, ?it/s]


AttributeError: 'NetworkSimulation' object has no attribute 'baseline_simulation'

In [ ]:
with open(f"../results/{network_name}/{datetime.date.today()}-{network_name}-test.json", 'w') as file:
    json.dump(simulation_logs, file, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)

# Analyse simulation logs

In [ ]:
df_heu = pd.DataFrame(simulation_logs)

In [ ]:
df_heu.query("latency_factor == 0.5 & vSDN_size == 2")[['hp_runtime', 'acceptance_ratio']]#.mean()

In [5]:
df_ilp = pd.DataFrame(simulation_logs)

In [6]:
df_ilp.query("latency_factor == 0.4 & vSDN_size == 3")[['hp_runtime', 'acceptable_count','acceptance_ratio', 'acceptance_ratio_ilp', 'h_list']]#.mean()

,hp_runtime,acceptable_count,acceptance_ratio,acceptance_ratio_ilp,h_list
1,0.73,36,0.580645,None,"[3, 6, 12, 15, 18, 19, 20, 23]"
9,0.61,33,0.532258,None,"[3, 6, 12, 15, 18, 19, 20, 23]"
17,0.64,32,0.516129,None,"[3, 6, 12, 15, 18, 19, 20, 23]"
25,0.69,31,0.500000,None,"[3, 6, 12, 15, 18, 19, 20, 23]"


In [ ]:
df_opt = pd.DataFrame(simulation_logs)

In [ ]:
df_opt.query("latency_factor == 0.4 & vSDN_size == 2")[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_ilp']]#.mean()